# **Homework 01**

### Q1. Running Elastic
Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:
```
curl localhost:9200
```
What's the `version.build_hash` value?

The result of `curl localhost:9200`:
```
{
  "name" : "5e31a9ee868c",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "k9nR8F9fQgO7cJlECNbD9w",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}
```

### Answer 1:
    "42f05b9372a9a4a470db3b52817899b99a76ee73"

---

## **Getting the data**

In [3]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:
```
pip install elasticsearch
```
Which function do you use for adding your data to elastic?

- `insert`
- `index`
- `put`
- `add`

### Answer 2:
    "index"

---

### Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

84.50
64.50
44.50
24.50
Look at the `_score` field.

In [4]:
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch('http://localhost:9200')

In [10]:
index_name = "course-questions"

if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [13]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

query = 'How do execute a command on a Kubernetes pod?'

/home/codespace/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████| 948/948 [00:03<00:00, 268.64it/s]


In [24]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    top_score = response['hits']['hits'][0]['_score']

    return top_score

In [25]:
elastic_search(query)

44.50556

### Answer 3:
    "44.5"

---

### Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.  

Return 3 results. What's the 3rd question returned by the search engine?

In [28]:
query = "How do copy a file to a Docker container?"

def elastic_search_q4(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        },
        "_source": ["question"]
    }

    response = es_client.search(index=index_name, body=search_query)

    # for q4
    # questions = [hit['_source']['question'] for hit in response['hits']['hits']]
    
    # for q5
    questions = response['hits']['hits']
    return questions

elastic_search_q4(query)

['How do I debug a docker container?',
 'How do I copy files from my local machine to docker container?',
 'How do I copy files from a different folder into docker container’s working directory?']


### Answer 4: The 3rd question returned by the search engine is:
    "How do I copy files from a different folder into docker container’s working directory?"

---

### Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)
```
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:
```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```
What's the length of the resulting prompt? (use the len function)

946
1446
1946
2446

In [33]:
query = "How do copy a file to a Docker container?"

def elastic_search_q5(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

records = elastic_search_q5(query)
records

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [40]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_entries = []
for record in records:
    context_entries.append(context_template.format(question=record['question'], text=record['text']))
context = "\n\n".join(context_entries)

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

question = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=question, context=context)

prompt_length = len(prompt)
print(prompt_length)

1462



### Answer 5:
    "1446"

---

### Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

```
pip install tiktoken
```
Let's calculate the number of tokens in our query:
```
encoding = tiktoken.encoding_for_model("gpt-4o")
```
Use the encode function. How many tokens does our prompt have?

120
220
320
420
Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

```
encoding.decode_single_token_bytes(63842)
```

```Python
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")

tokens = encoding.encode(prompt)
num_tokens = len(tokens)
print(f"The prompt contains {num_tokens} tokens")
```

### Answer 6:
    "220"

---